# Modeling Walkthrough

## Agenda

The goal here is to illustrate a possible workflow for classification modeling with `sklearn`'s `LogisticRegression` model.

SWBAT:

- formulate and implement an iterative modeling workflow.

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.impute import MissingIndicator, SimpleImputer

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import SelectFromModel

# plot_confusion_matrix is a handy visual tool, added in the latest version of scikit-learn
# if you are running an older version, comment out this line and just use confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_roc_curve

## Modeling Steps

1. Build a model based on the [Titanic dataset](https://www.kaggle.com/c/titanic/data) that predicts whether a given person survived or not
2. Evaluate the performance of the model
3. Make changes in an attempt to improve the model
4. Demonstrate whether an improvement was made

## The Data

This dataset has the following columns:

| Variable | Definition | Key |
| -------- | ---------- | --- |
| survival | Survival | 0 = No, 1 = Yes |
| pclass | Ticket class | 1 = 1st, 2 = 2nd, 3 = 3rd |
| sex | Sex | |
| Age | Age in years | |
| sibsp | # of siblings / spouses aboard the Titanic | |
| parch | # of parents / children aboard the Titanic | |
| ticket | Ticket number | |
| fare | Passenger fare | |
| cabin | Cabin number | |
| embarked | Port of Embarkation | C = Cherbourg, Q = Queenstown, S = Southampton |

## Initial Data Understanding and Preparation

Open up the file, get everything into `X` features and `y` target variables, divided into train and test.

In [ ]:
df = pd.read_csv("titanic.csv")

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.isna().sum()['Age']

Age data is missing for about 1 in 9 rows in our dataset.  For now, let's just exclude it, plus the non-numeric columns, and `PassengerId`, which doesn't seem like a real feature, but rather just an artifact of the dataset.

In [ ]:
df = df.drop("PassengerId", axis=1)

In [ ]:
df.dtypes

In [ ]:
# Pclass are numbers but it's not clear that the difference between 1st and 2nd is the
# same as the difference between 2nd and 3rd
numeric_columns = ["Survived", "SibSp", "Parch", "Fare"]

In [ ]:
sns.pairplot(df[numeric_columns]);

In [ ]:
numeric_df = df[numeric_columns]
X = numeric_df.drop("Survived", axis=1)
y = numeric_df["Survived"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2021)

## Modeling

Let's start with a completely "dummy" model, that will always choose the majority class.

In [ ]:
dummy_model = DummyClassifier(strategy="most_frequent")

Fit the model on our data

In [ ]:
dummy_model.fit(X_train, y_train)

We should expect all predictions to be the same

In [ ]:
# just grabbing the first 50 to save space
dummy_model.predict(X_train)[:50]

## Model Evaluation

In [ ]:
cross_val_score(dummy_model, X_train, y_train, cv=3)

So, the mean accuracy is a little over 62% if we always guess the majority class.

In [ ]:
fig, ax = plt.subplots()

fig.suptitle("Dummy Model")

plot_confusion_matrix(dummy_model, X_train, y_train, ax=ax, cmap="plasma");

In [ ]:
# just the numbers (this should work even with older scikit-learn)
confusion_matrix(y_train, dummy_model.predict(X_train))

A pretty lopsided confusion matrix!

## Modeling, Part 2

Let's use a logistic regression and compare its performance.

In [ ]:
simple_logreg_model = LogisticRegression(random_state=2021)

In [ ]:
simple_logreg_model.fit(X_train, y_train)

Look at the predictions:

In [ ]:
simple_logreg_model.predict(X_train)[:50]

Mixture of 1s and 0s this time

## Model Evaluation, Part 2

In [ ]:
cross_val_score(simple_logreg_model, X_train, y_train, cv=3)

So the mean accuracy is closer to 70% if the model is actually taking in information from the features instead of always guessing the majority class.

In [ ]:
fig, ax = plt.subplots()

fig.suptitle("Logistic Regression with Numeric Features Only")

plot_confusion_matrix(simple_logreg_model, X_train, y_train, ax=ax, cmap="plasma");

In [ ]:
confusion_matrix(y_train, simple_logreg_model.predict(X_train))

So, in general we are not labeling many of the "not survived" passengers as "survived", but for "survived" passengers we're getting it wrong most of the time.

## Data Preparation, Part 2

Maybe there is some useful information in the features we are not using yet.  Let's go wild and add all of them!

Note: you can and should add features incrementally in a "real" modeling context.  The engineering effort of encoding the variables can be non-trivial!  But here let's assume that it's not too much work to encode all of them.

Start with a new train-test split that contains all of the features

In [ ]:
X = df.drop("Survived", axis=1)
y = df["Survived"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2021)

In [ ]:
X_train.columns

In [ ]:
X_train.isna().sum()

### Handling Missing Values

Let's be extra cautious and make a separate column to indicate whether there originally was a missing value.

In our training data there are only missing values for a couple of the columns, but we can't be sure about where the test set will be missing data.

The `MissingIndicator` from `sklearn` will mark the missing values in an input array.

In [ ]:
indicator_demo = MissingIndicator()

indicator_demo.fit(X_train)

indicator_demo.features_

In [ ]:
indicator_demo.transform(X_train)[:5, :]

In [ ]:
X_train.iloc[:5, [3, 8, 9]]

In [ ]:
indicator = MissingIndicator(features="all")
indicator.fit(X_train)

In [ ]:
def add_missing_indicator_columns(X, indicator):
    """
    Helper function for transforming features
    
    For every feature in X, create another feature indicating whether that feature
    is missing. (This doubles the number of columns in X.)
    """
    
    # create a 2D array of True and False values indicating whether a given feature
    # is missing for that row
    missing_array_bool = indicator.transform(X)
    
    # transform into 1 and 0 for modeling
    missing_array_int = missing_array_bool.astype(int)
    
    # helpful for readability but not needed for modeling
    missing_column_names = [col + "_missing" for col in X.columns]
    
    # convert to df so it we can concat with X
    missing_df = pd.DataFrame(missing_array_int, columns=missing_column_names, index=X.index)
    
    return pd.concat([X, missing_df], axis=1)

In [ ]:
X_train = add_missing_indicator_columns(X=X_train, indicator=indicator)

In [ ]:
X_train.head()

Now that we've specified which values were originally missing, let's fill in those missing values.  This takes two separate imputers because we want to use the mean for numeric data and the majority class for categorical data.

The `SimpleImputer` class fills in the mean value by default, so we'll have to override that for the categorical columns.

In [ ]:
numeric_feature_names = ["Age", "SibSp", "Parch", "Fare"]
categorical_feature_names = ["Pclass", "Name", "Sex", "Ticket", "Cabin", "Embarked"]

X_train_numeric = X_train[numeric_feature_names]
X_train_categorical = X_train[categorical_feature_names]

In [ ]:
numeric_imputer = SimpleImputer()
numeric_imputer.fit(X_train_numeric)

In [ ]:
categorical_imputer = SimpleImputer(strategy="most_frequent")
categorical_imputer.fit(X_train_categorical)

We'll build a function here to minimize our work of imputation:

In [ ]:
def impute_missing_values(X, imputer):
    """
    Given a DataFrame and an imputer, use the imputer to fill in all
    missing values in the DataFrame
    """
    imputed_array = imputer.transform(X)
    imputed_df = pd.DataFrame(imputed_array, columns=X.columns, index=X.index)
    return imputed_df

In [ ]:
X_train_numeric = impute_missing_values(X_train_numeric, numeric_imputer)
X_train_categorical = impute_missing_values(X_train_categorical, categorical_imputer)

Double-check to make sure that all of the missing values are gone:

In [ ]:
X_train_imputed = pd.concat([X_train_numeric, X_train_categorical], axis=1)
X_train_imputed.isna().sum()

In [ ]:
X_train_imputed.head()

Drop all of the old columns from X_train, then concat the new imputed ones:

In [ ]:
X_train = X_train.drop(numeric_feature_names + categorical_feature_names, axis=1)
X_train = pd.concat([X_train_imputed, X_train], axis=1)

In [ ]:
X_train.head()

In [ ]:
X_train.isna().sum()

### One-Hot Encoding

Now that there are no missing values, convert all of the categorical features into numbers.

In [ ]:
def encode_and_concat_feature_train(X_train, feature_name):
    """
    Helper function for transforming training data.  It takes in the full X dataframe and
    feature name, makes a one-hot encoder, and returns the encoder as well as the dataframe
    with that feature transformed into multiple columns of 1s and 0s
    """
    # make a one-hot encoder and fit it to the training data
    ohe = OneHotEncoder(categories="auto", handle_unknown="ignore")
    single_feature_df = X_train[[feature_name]]
    ohe.fit(single_feature_df)
    
    # call helper function that actually encodes the feature and concats it
    X_train = encode_and_concat_feature(X_train, feature_name, ohe)
    
    return ohe, X_train

In [ ]:
def encode_and_concat_feature(X, feature_name, ohe):
    """
    Helper function for transforming a feature into multiple columns of 1s and 0s. Used
    in both training and testing steps.  Takes in the full X dataframe, feature name, 
    and encoder, and returns the dataframe with that feature transformed into multiple
    columns of 1s and 0s
    """
    # create new one-hot encoded df based on the feature
    single_feature_df = X[[feature_name]]
    feature_array = ohe.transform(single_feature_df).toarray()
    ohe_df = pd.DataFrame(feature_array, columns=ohe.categories_[0], index=X.index)
    
    # drop the old feature from X and concat the new one-hot encoded df
    X = X.drop(feature_name, axis=1)
    X = pd.concat([X, ohe_df], axis=1)
    
    return X

In [ ]:
encoders = {}

for categorical_feature in categorical_feature_names:
    ohe, X_train = encode_and_concat_feature_train(X_train, categorical_feature)
    encoders[categorical_feature] = ohe

In [ ]:
encoders

In [ ]:
X_train.head()

In [ ]:
X_train.shape

This is...a ridiculous number of columns.  How did we end up with more columns than rows?

(Answer: each unique name and ticket number is now its own column)

## Modeling, Part 3

Let's run a logistic regression on our ridiculous number of columns:

In [ ]:
logreg_model = LogisticRegression(random_state=2021)
logreg_model.fit(X_train, y_train)

What happened there?  This solver had no problem before.

Answer: it wasn't able to find the minimum with this number of steps in gradient descent.

### Adjustments

Let's try a couple of stopgap measures to get the model to run.

#### More iterations

In [ ]:
logreg_model_more_iterations = LogisticRegression(random_state=2021, max_iter=1000)
logreg_model_more_iterations.fit(X_train, y_train)

#### More regularization

Remember that the `C` parameter is the inverse of the regularization strength.

In [ ]:
logreg_model_more_regularization = LogisticRegression(random_state=2021, C=0.01)
logreg_model_more_regularization.fit(X_train, y_train)

#### Higher tolerance

In [ ]:
logreg_model_higher_tolerance = LogisticRegression(random_state=2021, tol=50)
logreg_model_higher_tolerance.fit(X_train, y_train)

## Model Evaluation, Part 3

In [ ]:
fix, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 6))

axes[0].set_title("More Iterations")
axes[1].set_title("More Regularization")
axes[2].set_title("Higher Tolerance")

plot_confusion_matrix(logreg_model_more_iterations, X_train, y_train,
                      ax=axes[0], cmap="plasma")
plot_confusion_matrix(logreg_model_more_regularization, X_train, y_train,
                      ax=axes[1], cmap="plasma")
plot_confusion_matrix(logreg_model_higher_tolerance, X_train, y_train,
                      ax=axes[2], cmap="plasma");

In [ ]:
print(cross_val_score(logreg_model_more_iterations, X_train, y_train, cv=3))
print(cross_val_score(logreg_model_more_regularization, X_train, y_train, cv=3))
print(cross_val_score(logreg_model_higher_tolerance, X_train, y_train, cv=3))

In [ ]:
fig, ax = plt.subplots()

plot_roc_curve(logreg_model_more_iterations, X_train, y_train, 
               name='logreg_model_more_iterations', ax=ax)
plot_roc_curve(logreg_model_more_regularization, X_train, y_train, 
               name='logreg_model_more_regularization', ax=ax)
plot_roc_curve(logreg_model_higher_tolerance, X_train, y_train, 
               name='logreg_model_higher_tolerance', ax=ax)

## Data Preparation, Part 3

Let's scale all of the features, so the model isn't overly penalizing age and fare

In [ ]:
scaler = StandardScaler()

scaler.fit(X_train)

In [ ]:
def scale_values(X, scaler):
    """
    Given a DataFrame and a fitted scaler, use the scaler to scale all of the features
    """
    scaled_array = scaler.transform(X)
    scaled_df = pd.DataFrame(scaled_array, columns=X.columns, index=X.index)
    return scaled_df

In [ ]:
X_train = scale_values(X_train, scaler)

In [ ]:
X_train.head()

## Modeling, Part 4

Now that the data is scaled, let's see if we can fit the model without tweaking any hyperparameters.

In [ ]:
logreg_model = LogisticRegression(random_state=2021)
logreg_model.fit(X_train, y_train)

## Model Evaluation, Part 4

Now that we are able to run a logistic regression with default hyperparameters, let's see how that performs.

In [ ]:
fig, ax = plt.subplots()

fig.suptitle("Logistic Regression with All Features, Scaled")

plot_confusion_matrix(logreg_model, X_train, y_train, ax=ax, cmap="plasma");

In [ ]:
all_features_cross_val_score = cross_val_score(logreg_model, X_train, y_train, cv=3)
all_features_cross_val_score

In [ ]:
plot_roc_curve(logreg_model, X_train, y_train)

Perfect on the training data, high 70% range on the test data ... this might be overfitting.

In [ ]:
sorted(list(zip(X_train.columns, logreg_model.coef_[0])),
       key=lambda x: abs(x[1]), reverse=True)[:50]

## Hyperparameter Adjustment


### Different Regularization Strengths

Let's try out some different regularization penalties to see if we can improve the test data score a bit.

In [ ]:
logreg_model = LogisticRegression(random_state=2021, C=0.1)
logreg_model.fit(X_train, y_train)

print("Old:", all_features_cross_val_score)
print("New:", cross_val_score(logreg_model, X_train, y_train, cv=3))

Seems like it doesn't make a difference.

Try a little less regularization:

In [ ]:
logreg_model = LogisticRegression(random_state=2021, C=0.5)
logreg_model.fit(X_train, y_train)

print("Old:", all_features_cross_val_score)
print("New:", cross_val_score(logreg_model, X_train, y_train, cv=3))

Same or worse.

Try a little more regularization:

In [ ]:
logreg_model = LogisticRegression(random_state=2021, C=0.05)
logreg_model.fit(X_train, y_train)

print("Old:", all_features_cross_val_score)
print("New:", cross_val_score(logreg_model, X_train, y_train, cv=3))

Also worse.  It looks like the default C value is pretty optimal for this solver.

### Different Solvers

Let's try also some other solvers:

In [ ]:
logreg_model = LogisticRegression(random_state=2021, solver="liblinear")
logreg_model.fit(X_train, y_train)

print("Old:", all_features_cross_val_score)
print("New:", cross_val_score(logreg_model, X_train, y_train, cv=3))

A little slower, but no major difference in the scores.  Let's try adding some more regularization:

In [ ]:
logreg_model = LogisticRegression(random_state=2021, solver="liblinear", C=0.01)
logreg_model.fit(X_train, y_train)

print("Old:", all_features_cross_val_score)
print("New:", cross_val_score(logreg_model, X_train, y_train, cv=3))

Getting better.  Try a different type of penalty:

In [ ]:
logreg_model = LogisticRegression(random_state=2021, solver="liblinear", penalty="l1")
logreg_model.fit(X_train, y_train)

print("Old:", all_features_cross_val_score)
print("New:", cross_val_score(logreg_model, X_train, y_train, cv=3))

Slightly better average here.  Try adding some more regularization with L1 penalty:

In [ ]:
logreg_model = LogisticRegression(random_state=2021, solver="liblinear", penalty="l1", C=0.01)
logreg_model.fit(X_train, y_train)

print("Old:", all_features_cross_val_score)
print("New:", cross_val_score(logreg_model, X_train, y_train, cv=3))

Still, the default regularization strength seems pretty good.  Double-check the confusion matrix:

In [ ]:
logreg_model = LogisticRegression(random_state=2021, solver="liblinear", penalty="l1")
logreg_model.fit(X_train, y_train)

fig, ax = plt.subplots()

fig.suptitle("Logistic Regression with All Features (Scaled, Hyperparameters Tuned)")

plot_confusion_matrix(logreg_model, X_train, y_train, ax=ax, cmap="plasma");

## `SelectFromModel`

The last model is probably overfitting. We might try thinning out the number of features by eliminating the ones with small modeling coefficients:

In [ ]:
selector = SelectFromModel(logreg_model)

selector.fit(X_train, y_train)

We're using the default threshold here:

In [ ]:
thresh = selector.threshold_
thresh

Let's get a sense of which features will be eliminated:

In [ ]:
coefs = selector.estimator_.coef_
coefs

In [ ]:
coefs[coefs > thresh].shape

In [ ]:
selector.get_support()

In [ ]:
dict(zip(X_train.columns, selector.get_support()))

In [ ]:
def select_important_features(X, selector):
    """
    Given a DataFrame and a selector, use the selector to choose
    the most important columns
    """
    imps = dict(zip(X.columns, selector.get_support()))
    selected_array = selector.transform(X)
    selected_df = pd.DataFrame(selected_array,
                               columns=[col for col in X.columns if imps[col]],
                               index=X.index)
    return selected_df

In [ ]:
X_train_selected = select_important_features(X=X_train, selector=selector)

In [ ]:
logreg_sel = LogisticRegression(random_state=2021, solver="liblinear", penalty="l1")

logreg_sel.fit(X_train_selected, y_train)

In [ ]:
cross_val_score(logreg_sel, X_train_selected, y_train, cv=3)

Probably still overfitting, but let's call this our final model!

## Final Model Evaluation

Now that we have a final model, run X_test through all of the preprocessing steps so we can evaluate the model's performance

In [ ]:
X_test_no_transformations = X_test.copy()

In [ ]:
# add missing indicators
X_test_mi = add_missing_indicator_columns(X_test_no_transformations, indicator)

In [ ]:
# separate out values for imputation
X_test_numeric = X_test_mi[numeric_feature_names]
X_test_categorical = X_test_mi[categorical_feature_names]

In [ ]:
# impute missing values
X_test_numeric = impute_missing_values(X_test_numeric, numeric_imputer)
X_test_categorical = impute_missing_values(X_test_categorical, categorical_imputer)
X_test_imputed = pd.concat([X_test_numeric, X_test_categorical], axis=1)
X_test_new = X_test_mi.drop(numeric_feature_names + categorical_feature_names, axis=1)
X_test_final = pd.concat([X_test_imputed, X_test_new], axis=1)

In [ ]:
# one-hot encode categorical data
for categorical_feature in categorical_feature_names:
    X_test_final = encode_and_concat_feature(X_test_final,
                                       categorical_feature, encoders[categorical_feature])

In [ ]:
# scale values
X_test_scaled = scale_values(X_test_final, scaler)

In [ ]:
# select features
X_test_selected = select_important_features(X_test_scaled, selector)

In [ ]:
X_test_selected.head()

Create a model with the relevant hyperparameters, fit, and score

In [ ]:
final_model = LogisticRegression(random_state=2021, solver="liblinear", penalty="l1")
final_model.fit(X_train_selected, y_train)

final_model.score(X_test_selected, y_test)

### Compare the past models

In [ ]:
# Create a way to categorize our different models
model_candidates = [
    {
        'name':'logreg_model_more_iterations'
        ,'model':logreg_model_more_iterations
        ,'X_test':X_test_final
        ,'y_test':y_test
    },
    {
        'name':'logreg_model_more_regularization'
        ,'model':logreg_model_more_regularization
        ,'X_test':X_test_final
        ,'y_test':y_test
    },
    {
        'name':'logreg_model_higher_tolerance'
        ,'model':logreg_model_higher_tolerance
        ,'X_test':X_test_final
        ,'y_test':y_test
    },
    {
        'name':'dummy_model'
        ,'model':dummy_model
        ,'X_test':X_test
        ,'y_test':y_test
    },
    {
        'name':'simple_logreg_model'
        ,'model':simple_logreg_model
        ,'X_test':X_test_no_transformations[["SibSp", "Parch", "Fare"]]
        ,'y_test':y_test
    },
    {
        'name':'final_model'
        ,'model':final_model
        ,'X_test':X_test_selected
        ,'y_test':y_test
    }
]

In [ ]:
final_scores_dict = {
    "Model Name": [candidate.get('name') for candidate in model_candidates],
    "Mean Accuracy": [
        candidate.get('model').score(
                                candidate.get('X_test'), 
                                candidate.get('y_test')
        ) 
        for candidate in model_candidates
    ]
    
}
final_scores_df = pd.DataFrame(final_scores_dict).set_index('Model Name')
final_scores_df

Final comparison of confusion matrices

In [ ]:
nrows = 2
ncols = round(len(model_candidates)/nrows)

fig, axes = plt.subplots(
                nrows=nrows,
                ncols=ncols,
                figsize=(12, 6)
)

fig.suptitle("Confusion Matrix Comparison")


for i,candidate in enumerate(model_candidates):
    # Logic for making rows and columns for matrices
    row = i // 3
    col = i % 3
    ax = axes[row][col]
    
    ax.set_title(candidate.get('name'))
    plot_confusion_matrix(
        candidate.get('model'),
        candidate.get('X_test'),
        candidate.get('y_test'),
        ax=ax,
        cmap="plasma"
    )
    
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots()

# Plot only the last models we created (so it's not too cluttered)
for model_candidate in model_candidates[3:]:
    plot_roc_curve(
        model_candidate.get('model'),
        model_candidate.get('X_test'),
        model_candidate.get('y_test'), 
        name=model_candidate.get('name'),
        ax=ax
    )

In [ ]:
fig, ax = plt.subplots()

# Plot the final model against the other earlier models
plot_roc_curve(
    final_model, 
    X_test_selected, 
    y_test,
    name='final_model', 
    ax=ax
)

for model_candidate in model_candidates[:3]:
    plot_roc_curve(
        model_candidate.get('model'),
        model_candidate.get('X_test'),
        model_candidate.get('y_test'), 
        name=model_candidate.get('name'),
        ax=ax
    )

# Exercise

Build and iterate on a logistic regression model of **color** for the diamonds dataset! Maximize accuracy.

In [ ]:
diamonds = sns.load_dataset('diamonds')